In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'carnegieendowment_org'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
publish_source = 'carnegieendowment.org'
country = 'United States'
language = 'English'
images_path = []

cat_pages = c.download_page('https://carnegieendowment.org/sada')
cat_pages = c.scrap('<ul\s*class="clean-list list-across">(.*?)<a\s*class="ar no-text-transform no-letter-spacing"',cat_pages)
for _ in cat_pages.split('<li>')[1:]:
    
    cat_url = c.scrap('href="(.*?)"', _)
    if 'http' not in cat_url:
        cat_url = 'https://carnegieendowment.org' + cat_url
    logger.info(f'Fetching cat url {cat_url}\n')
    cat_page = c.download_page(cat_url)
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue

    for i in cat_page.split('<div class="overflow-hidden">')[1:]:

        # source_link
        source_link = c.scrap('<a href="(.*?)"', i)
        if 'http' not in source_link:
            source_link = 'https://carnegieendowment.org' + source_link
        if '/sada/experts' in source_link:
            continue
        if 'https://itunes.apple.com/us/app/sada/id723205755' in source_link:
            continue
       
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))
        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_article_url += 1
                continue 
       
        # source_headline
        source_headline = c.scrap('<meta\s*name="title" content="(.*?)"', page)

        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('datePublished":\s*"(.*?)GMT\+0000', page)   # Fri Mar 22 2019 10:27:38
            date_time_obj = datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')
            # Singapore is 2 hours and 30 minutes ahead of India
            ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  # utc+8 to ist
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass

        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
#         if pub_date != created_on:
#             break    

        logger.info(f'Fetching {source_link}\n')  

        # source_content          
        source_content = c.scrap('<div\s*class="article-body first-letter">(.*?)<div\s*class="section divider divider--slim">', page)
        source_content = c.strip_html(source_content)

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist =  c.scrap('"author":.*?"name":\s*"(.*?)"',page)

        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')


<li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1589">Algeria Ahead of Elections</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1327">Egypt in Search of Balance</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/2737">Sada Feature</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1325">Syria in Crisis</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1326">Tunisia in Transition</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1499">Militias and the Future of the Iraqi State</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1501">Refugees’ Plight Across the Region</a></div></li><li><div class="gutter-half-bottom"><a class="area-focus" href="/sada/issue/1381">Multimedia Series</a></div></li></ul></div></div></li><li><a hr

SyntaxError: 'continue' not properly in loop (<ipython-input-1-5727508b684b>, line 50)

50


shish
